# LEE Analyzer plots


## Imports & Constants

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

pd.options.display.max_columns = 999
gr      = 1.618

mass_p= 0.93827 #GeV
mass_e= 0.00511 #GeV
fid_min=10

In [2]:
# TPC border check
def CheckBorderTPC(x,y,z,tolerance=0):
    detectorx   =256.35     # In cm
    detectory   =116.5      # Symmetric around 0     
    detectorz   =1036.8
    d=tolerance # border tolerance
    if (0+d) < x < (detectorx-d):
            if (-detectory+d)< y < (detectory-d):
                    if (0+d) < z < (detectorz-d):
                        return True
    return False


# True shower energy containment graphs
def effErr(teller,noemer):
    return np.sqrt(teller*(1-teller/noemer))/noemer


# Summed true shower energy, not corrected for electron mass
def true_showerE(row):
    index= np.where( (row['nu_daughters_pdg']==11) | (row['nu_daughters_pdg']==22) )
    E_true = sum(row['nu_daughters_E'][index])
    return E_true

def TrackRecoEnd(row):
    for x,y,z in zip(row['track_end_x'],row['track_end_y'],row['track_end_z']):
        if not CheckBorderTPC(x,y,z,tolerance=fid_min):
            return False
    return True

# Sums hits over the shower objects
def shower_nhits_plane(shower_nhits):
    sh_hits_plane = [0,0,0,0]
    for sh in shower_nhits:
        sh_hits_plane[:-1]+=sh
    sh_hits_plane[-1]=max(sh_hits_plane[:-1])
    return sh_hits_plane

def categoryPlus(row):
    if row['nu_pdg']==12:
        if row['category']=='e CC':
            return r'$\nu_e$ CC       '
        elif row['category']=='Mixed':
            return r'$\nu_e$ cont.   '
        else:
            return row['category']+' '*(10-len(row['category']))
    
    elif row['nu_pdg']==14:
        if row['category']=='mu CC':
            return r'$\nu_{\mu}$ CC       '
        elif row['category']=='Mixed':
            return r'$\nu_{\mu}$ cont.   '
        else:
            return row['category']+' '*(10-len(row['category']))
        
    elif row['nu_pdg']==4294967282 or row['nu_pdg']==4294967284:
        return r'$\bar{\nu}_{\mu} / \bar{\nu}_e$     '
    else:
        print (row['nu_pdg'])
        return 'Cosmic intime '
    
# Formatting
def sciNot(x):
    x=float(x)
    return "{0:4.1f}".format(x)

## Load Pickle file

In [3]:
df_shower = pd.read_pickle('Input/nu/nu_cosmic_passed.pckl')
print('Dataframe loaded from Pickle has '+str(len(df_shower.index))+' entries.' )

columns_remove = ['matched_tracks', 'nu_daughters_endx', 'track_start_x', 'track_nhits',
       'matched_showers', 'shower_dir_y', 'track_start_y', 'track_end_x',
       'shower_dEdx', 'nu_daughters_E', 'true_vz_sce',
       'predict_mu', 'vy', 'bnbweight', 'shower_start_y', 'shower_nhits',
       'shower_energy', 'shower_length', 'vx', 'shower_start_x',
       'matched_tracks_energy', 'shower_dir_z', 'nu_daughters_endz',
       'track_end_z', 'vz', 'true_shower_depE', 'nu_daughters_pz',
       'predict_pi', 'predict_p', 'true_shower_z_sce', 'track_energy_dedx',
       'track_start_z', 'track_dir_y', 'matched_showers_energy',
       'nu_daughters_endy', 'track_dEdx', 'track_dir_z', 'true_shower_x_sce',
       'track_pca', 'track_dir_x', 'track_energy_hits',
       'predict_cos', 'shower_dEdx_hits', 'track_dEdx_hits',
       'true_shower_y_sce', 'nu_daughters_py', 'true_vy_sce',
       'true_vx_sce', 'nu_daughters_px', 'shower_dir_x', 'predict_em',
       'track_end_y', 'true_shower_pdg', 'shower_pca',
       'shower_open_angle', 'shower_start_z', 'true_shower_endx',
       'true_shower_endy', 'true_shower_endz', 'true_shower_tmax_x',
       'true_shower_tmax_y', 'true_shower_tmax_z', 'e_contained',
       'e_contained_tmax', 'reco_length_containment',
       'shower_daughter', 'track_daughter', 'plane2', 'total_nhits']
df_shower.drop(columns_remove,axis=1,inplace=True)
df_shower.head(3)

Dataframe loaded from Pickle has 113345 entries.


,distance,nu_daughters_pdg,nu_pdg,nu_E,category,shower_containment_q,shower_sp_profile,reconstructed_energy
0,255.623115,"[13, 2212, 2112, 2000000101, 2000000101]",14,1.184815,Cosmic,0.999995,0.050590,"[0.224278519611, 0.360606951127, 0.31402080667..."
1,458.972726,"[13, 2212, 2212]",14,0.863875,Cosmic,0.999996,1.177586,"[0.0446746716026, 0.0679343334161, 0.064011868..."
2,34.671440,"[13, 2212, 211, 2112]",14,1.742921,mu CC,0.999999,0.349142,"[0.256887470396, 0.44006767008, 0.487820845319..."


In [4]:
#df_shower['collectionhits']=df_shower['shower_nhits'].apply(lambda x: sum([sh[2] for sh in x]))
print(df_shower.columns)
print(len(df_shower[(df_shower['category'].isin(['mu CC'])) & (df_shower['distance']<5) ].index))
print(len(df_shower))

Index(['distance', 'nu_daughters_pdg', 'nu_pdg', 'nu_E', 'category',
       'shower_containment_q', 'shower_sp_profile', 'reconstructed_energy'],
      dtype='object')
33630
113345


## Plots

In [5]:
# Plot of the selected events in true neutrino energy:
df_shower['catplus']=df_shower.apply(categoryPlus,axis=1)
df = df_shower
df['reco_E']=df['reconstructed_energy'].apply(lambda x: x[-1])

categories = df['catplus'].unique()
true_E = []
reco_E = [] 
labels = []

for cat in categories:
    true_E.append(df[df['catplus']==cat]['nu_E'])
    reco_E.append(df[df['catplus']==cat]['reco_E'])
    #print(cat)
    lab = cat+ '('+sciNot(len(true_E[-1])/entries*100) +'%)'
    labels.append(lab)
    

NameError: name 'entries' is not defined

In [ ]:
plt.style.use('default')
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(4.3*gr,4),sharey=True,sharex=True)
#plt.suptitle(r'Passed events in True and Reconstructed $\nu$ Energy')
N = 50
min_x = 0.0
max_x = 2.5
step  = 0.5
entries= len(df.index)

ax[0].hist(true_E,N,range=(min_x,max_x), stacked=True, fill=True,label=labels,normed=True)
ax[1].hist(reco_E,N,range=(min_x,max_x), stacked=True, fill=True,label=labels,normed=True)

ax[0].set_ylabel('Area normalised ('+sciNot((max_x-min_x)/N)+' GeV bins )')
ax[0].set_xlabel(r'True $\nu$ energy [GeV]')
ax[1].set_xlabel(r'Reco $\nu$ energy [GeV]')

plt.xticks(np.arange(min_x,max_x+step, step))
plt.legend(ncol=3,loc='upper center',bbox_to_anchor=(-0.1, 1.26))

ax[0].set_frame_on(False)
ax[1].set_frame_on(False)
ax[1].text(1.25,1.0,r'$1.48\times 10^{21}$ POT')

#ax[0].spines['top'].set_visible(False)
#ax[0].spines['right'].set_visible(False)
#ax[1].spines['top'].set_visible(False)
#ax[1].spines['right'].set_visible(False)

plt.savefig("BNB_mu.png",dpi=500,
            transparant=True,
               bbox_inches='tight', 
               pad_inches=0.1)

In [ ]:
# Track length


def track_length(row):
    length=[]
    looplist= zip(row['track_start_x'],row['track_start_y'],row['track_start_z'],
               row['track_end_x'],row['track_end_y'],row['track_end_z'])
    
    for x,y,z,e_x,e_y,e_z in looplist:
        length.append(np.linalg.norm( [x-e_x,y-e_y,z-e_z]   ))
    return length



series_obj = df_shower.apply(track_length,axis=1)
lengths =np.hstack( series_obj.values )
pdgs = np.hstack( df_shower['matched_tracks'].values )


lengths_e = lengths[np.where(pdgs==11)]
lengths_m = lengths[np.where(abs(pdgs)==13)]
lengths_p = lengths[np.where(pdgs==2212)]
lengths_n = lengths[np.where(pdgs==2112)]
lengths_g = lengths[np.where(pdgs==22)]
lengths_pi = lengths[np.where(abs(pdgs)==211)]


data=[lengths_p,lengths_m,lengths_e,lengths_pi,lengths_g,lengths_n]
labels = ['Proton (30167)',r'$\mu^-/\mu^+$ (14014)','Electron (9837)',r'$\pi^-/\pi^+$ (9210)','Gamma (1438)','Neutron (1363)']

ax[0].hist(data, N,range=(0,200), histtype='step', stacked=False, fill=False,label=labels)
ax[1].hist(data, N,range=(0,200), histtype='step',normed=True, stacked=False, fill=False,label=labels)
#ax[0].axvline(x=150,alpha=.2,color='k',label='Proposed cut')
#ax[1].axvline(x=150,alpha=.2,color='k',label='Proposed cut')

ax[0].legend()
ax[0].set_ylabel('#Events')
ax[0].set_xlabel('Tracklength [cm]')

ax[1].set_ylabel('Normalised')
ax[1].set_xlabel('Tracklength [cm]')

handles, labels = ax[0].get_legend_handles_labels()
ax[0].legend(handles[::-1], labels[::-1],loc='upper right')

plt.tight_layout()
fig.subplots_adjust(top=0.88)
plt.savefig('tracklength.pdf')

unique, counts = np.unique(pdgs, return_counts=True)
print(dict(zip(unique, counts)))

cut=150
print(sum(lengths_p>cut))
print(sum(lengths_m>cut))
print(sum(lengths_e>cut))
print(sum(lengths_pi>cut))
print(sum(lengths_g>cut))
print(sum(lengths_n>cut))
    